In [5]:
# In a notebook cell, prefix with `!` to run Streamlit in a subprocess:
!streamlit run "/Volumes/TAFOL/Daniel Cooke/Work & Learning/Learning/Python/Streamlit/Quiddler-ScoreSheet/quiddler.py"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.10.179:8501

^C
  Stopping...
Exception ignored in: <module 'threading' from '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py'>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1622, in _shutdown
    lock.acquire()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/streamlit/web/bootstrap.py", line 43, in signal_handler
    server.stop()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/streamlit/web/server/server.py", line 458, in stop
    self._runtime.stop()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/streamlit/runtime/runtime.py", line 324, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/Library/Framewo

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np # Import numpy for NaN if needed, though pandas handles None well

class QuiddlerScoresheet:
    """Interactive score sheet for Quiddler card game using Streamlit."""

    def __init__(self):
        self._initialize_session_state()

    def _initialize_session_state(self):
        """Initialize session state variables with defaults."""
        defaults = {
            "num_players": 2,
            "num_games": 5,
            "settings_changed": False,
            "df_scores": None # Initialize df_scores as None, it will be populated later
        }
        
        for key, value in defaults.items():
            if key not in st.session_state:
                st.session_state[key] = value
        
        # Ensure player names are initialized in session state
        for i in range(defaults["num_players"]):
            if f"player_name_{i}" not in st.session_state:
                st.session_state[f"player_name_{i}"] = f"Player {i + 1}"

    def _get_player_names(self):
        """Get current player names from session state."""
        # Use the actual player names stored in session_state, not just defaults
        return [
            st.session_state.get(f"player_name_{i}", f"Player {i + 1}")
            for i in range(st.session_state.num_players)
        ]

    def _create_empty_dataframe(self):
        """Create a new DataFrame with current settings."""
        player_names = self._get_player_names()
        return pd.DataFrame({
            "Round": list(range(1, st.session_state.num_games + 1)),
            **{name: [None] * st.session_state.num_games for name in player_names} # Use None for empty cells
        })

    def _preserve_existing_scores(self, old_df, new_df):
        """Copy scores from old DataFrame to new one where possible."""
        if old_df is None or old_df.empty:
            return new_df
            
        # Copy existing scores for matching columns and rows
        for col in new_df.columns:
            if col in old_df.columns and col != "Round":
                min_rows = min(len(new_df), len(old_df))
                # Ensure the data type is compatible for assignment
                # Convert old_df values to numeric type before copying to new_df
                new_df.loc[:min_rows-1, col] = old_df.loc[:min_rows-1, col].replace({None: np.nan}).astype(float).values
                
        return new_df

    def _needs_dataframe_rebuild(self):
        """Check if DataFrame needs to be rebuilt due to setting changes or initial load."""
        # If df_scores is None, it means it hasn't been initialized yet
        if st.session_state["df_scores"] is None:
            return True
            
        df = st.session_state["df_scores"]
        expected_cols = ["Round"] + self._get_player_names()
        
        # Check if columns or number of games (rows) have changed
        # or if a general setting change flag is set
        return (
            list(df.columns) != expected_cols or 
            len(df) != st.session_state.num_games or
            st.session_state.get("settings_changed", False)
        )

    def _update_scores_dataframe(self):
        """Update or create the scores DataFrame as needed."""
        if self._needs_dataframe_rebuild():
            old_df = st.session_state["df_scores"] # Get current df from session state
            new_df = self._create_empty_dataframe()
            
            # Preserve existing scores when rebuilding the DataFrame
            if old_df is not None:
                new_df = self._preserve_existing_scores(old_df, new_df)
            
            st.session_state["df_scores"] = new_df
            st.session_state["settings_changed"] = False # Reset flag after rebuild

    def render_settings(self):
        """Render game configuration controls."""
        st.markdown("### Game Settings")
        
        col1, col2 = st.columns(2)
        
        # Use current values from session state for number inputs
        current_num_players = st.session_state.num_players
        current_num_games = st.session_state.num_games

        with col1:
            new_players = st.number_input(
                "Number of players",
                min_value=2,
                max_value=8,
                value=current_num_players,
                help="How many people are playing?",
                key="num_players_input" # Unique key for the widget
            )
            
        with col2:
            new_games = st.number_input(
                "Number of rounds",
                min_value=1,
                max_value=10,
                value=current_num_games,
                help="How many rounds to play (max 10)",
                key="num_games_input" # Unique key for the widget
            )
        
        # Detect if settings have changed and update session state
        if new_players != current_num_players:
            st.session_state.num_players = new_players
            st.session_state.settings_changed = True
            # When number of players changes, ensure player names are initialized for new players
            for i in range(current_num_players, new_players):
                if f"player_name_{i}" not in st.session_state:
                    st.session_state[f"player_name_{i}"] = f"Player {i + 1}"
        
        if new_games != current_num_games:
            st.session_state.num_games = new_games
            st.session_state.settings_changed = True

    def render_player_names(self):
        """Render player name input fields."""
        st.markdown("### Player Names")
        
        cols = st.columns(st.session_state.num_players)
        names_changed_flag = False
        
        for i in range(st.session_state.num_players):
            with cols[i]:
                # Get the current player name from session state (or default if not set)
                current_name_in_state = st.session_state.get(f"player_name_{i}", f"Player {i + 1}")
                
                # Render text input. Streamlit automatically updates st.session_state[key]
                # when the user changes the value.
                input_value = st.text_input(
                    f"Player {i + 1}",
                    value=current_name_in_state, # Set initial value from session state
                    key=f"player_name_{i}",      # This key ensures direct update to st.session_state
                    placeholder=f"Player {i + 1}"
                )
                
                # Check if the value in session_state (updated by the widget) is different
                # from what it was *before* this run (represented by current_name_in_state).
                if input_value != current_name_in_state:
                    names_changed_flag = True
        
        if names_changed_flag:
            st.session_state.settings_changed = True # Signal that player names have changed

    def render_score_editor(self):
        """Render the interactive score table."""
        st.markdown("### Score Entry")
        
        # df_scores should already be correctly initialized/rebuilt by _update_scores_dataframe
        
        # Retrieve the DataFrame reference directly from session state
        df_for_editor = st.session_state["df_scores"]
        
        # Configure column display
        column_config = {
            "Round": st.column_config.TextColumn(
                "Round", 
                disabled=True,
                width="small"
            )
        }
        
        # Configure player columns to accept numbers
        for col in df_for_editor.columns:
            if col != "Round":
                column_config[col] = st.column_config.NumberColumn(
                    col,
                    min_value=0,
                    max_value=999,
                    step=1,
                    format="%d"
                )
        
        # Render data editor. By passing `st.session_state["df_scores"]` directly as `data`,
        # Streamlit automatically updates this session state variable when the user makes edits.
        st.data_editor(
            st.session_state["df_scores"], # Pass the session state variable directly
            use_container_width=True,
            num_rows="fixed",
            column_config=column_config,
            hide_index=True,
            key="score_editor" # Crucial for data_editor's internal state management
        )
        # No need to manually assign back to st.session_state here, Streamlit does it.

    def render_totals(self):
        """Display running totals for each player."""
        if "df_scores" not in st.session_state or st.session_state["df_scores"] is None:
            return

        df = st.session_state["df_scores"]
        player_cols = [col for col in df.columns if col != "Round"]
        
        if not player_cols:
            return

        # Calculate totals: Fill None with 0, then explicitly convert to numeric type before summing
        # This is the key change to ensure correct summation
        totals = df[player_cols].fillna(0).astype(int).sum()
        
        st.markdown("### Current Totals")
        
        # Create totals display with Streamlit metrics
        total_cols = st.columns(len(player_cols))
        for i, (player, total) in enumerate(zip(player_cols, totals)):
            with total_cols[i]:
                st.metric(
                    label=player,
                    value=int(total),
                    help=f"Total score for {player}"
                )

    def render_game_summary(self):
        """Display game summary and winner if all rounds completed."""
        if "df_scores" not in st.session_state or st.session_state["df_scores"] is None:
            return
            
        df = st.session_state["df_scores"]
        player_cols = [col for col in df.columns if col != "Round"]
        
        if not player_cols:
            return

        # Check if any scores have been entered yet (treating None as 0)
        # Apply fillna(0) and astype(int) for robust check
        if df[player_cols].fillna(0).astype(int).sum().sum() == 0:
            return
            
        # Calculate totals for summary: Fill None with 0, then explicitly convert to numeric type
        totals = df[player_cols].fillna(0).astype(int).sum()
        max_total = totals.max()
        winners = totals[totals == max_total].index.tolist()
        
        # Criteria for showing game status: at least half the cells have non-zero values
        # Need to ensure this check also uses numeric values after fillna
        non_zero_count = (df[player_cols].fillna(0).astype(int) != 0).sum().sum()
        total_cells = len(player_cols) * len(df)
        
        if total_cells > 0 and non_zero_count >= total_cells * 0.5:
            st.markdown("---") # Visual separator
            st.markdown("### 🏆 Game Status")
            if len(winners) == 1:
                st.success(f"**{winners[0]}** is currently winning with **{int(max_total)}** points!")
            else:
                winner_names = ", ".join(winners)
                st.info(f"**Tie** between {winner_names} with **{int(max_total)}** points!")

    def render_scoresheet(self):
        """Render the complete scoresheet interface."""
        
        # This is crucial: Ensure the DataFrame is updated/rebuilt BEFORE
        # any component (like data_editor) tries to read or display it.
        self._update_scores_dataframe() 

        # ---
        with st.expander("⚙️ Game Settings & Player Names", expanded=True):
            self.render_settings()
            st.divider()
            self.render_player_names()

        # ---
        st.divider()
        
        # Main score entry
        self.render_score_editor()
        
        # ---
        st.divider()
        
        # Totals and summary
        col1, col2 = st.columns([2, 1])
        with col1:
            self.render_totals()
        with col2:
            self.render_game_summary()

    def export_scores(self):
        """Export scores to CSV (future enhancement)."""
        if "df_scores" in st.session_state and st.session_state["df_scores"] is not None:
            # Fill None with empty strings for a cleaner CSV export
            # Ensure scores are treated as numeric for export (or just as is if okay)
            # If you want to export numbers, it's good to ensure numeric type for cells with data.
            # For simplicity, filling None with empty string is often fine for CSV
            return st.session_state["df_scores"].fillna('').to_csv(index=False)
        return None


def main():
    """Main application entry point."""
    st.set_page_config(
        page_title="Quiddler Score Sheet",
        page_icon="🃏",
        layout="centered",
        initial_sidebar_state="collapsed"
    )
    
    scoresheet = QuiddlerScoresheet()
    scoresheet.render_scoresheet()


if __name__ == "__main__":
    main()